In [1]:
import numpy as np
import tensorflow as tf

from kernelpooling import _generate_sketch_matrix

In [2]:
sketch_matrix = _generate_sketch_matrix(rand_h, rand_s, output_dim)

sess = tf.InteractiveSession()
smat = sketch_matrix.eval()
print(smat)
sess.close()

NameError: name 'rand_h' is not defined

In [135]:
import numpy as np
import tensorflow as tf

from math import factorial

from keras import backend as K
from keras.engine.topology import Layer


def _fft(bottom, sequential, compute_size):
    #if sequential:
    #    return sequential_batch_fft(bottom, compute_size)
    #else:
    return tf.fft(bottom)

def _ifft(bottom, sequential, compute_size):
    #if sequential:
    #    return sequential_batch_ifft(bottom, compute_size)
    #else:
    return tf.ifft(bottom)

def _generate_sketch_matrix(rand_h, rand_s, output_dim):
    """
    Return a sparse matrix used for tensor/count sketch operation,
    which is random feature projection from input_dim-->output_dim.
    
    Parameters
    ----------
    rand_h: array, shape=(input_dim,)
        Vector containing indices in interval `[0, output_dim)`. 
    rand_s: array, shape=(input_dim,)
        Vector containing values of 1 and -1.
    output_dim: int
        The dimensions of the count sketch vector representation.

    Returns
    -------
    sparse_sketch_matrix : SparseTensor
        A sparse matrix of shape [input_dim, output_dim] for count sketch.
    """

    # Generate a sparse matrix for tensor count sketch
    rand_h = rand_h.astype(np.int64)
    rand_s = rand_s.astype(np.float32)
    assert(rand_h.ndim==1 and rand_s.ndim==1 and len(rand_h)==len(rand_s))
    assert(np.all(rand_h >= 0) and np.all(rand_h < output_dim))

    input_dim = len(rand_h)
    indices = np.concatenate((np.arange(input_dim)[..., np.newaxis],
                              rand_h[..., np.newaxis]), axis=1)
    sparse_sketch_matrix = tf.sparse_reorder(
        tf.SparseTensor(indices, rand_s, [input_dim, output_dim]))
    return sparse_sketch_matrix

# Write hash initializer here...
def _count_sketch(sketch_matrix, x):
    '''Compute the Count Sketch for Taylor series kernel on input feature vectors `x`.

    Parameters
    ----------
    sketch_matrix : SparseTensor, shape [input_dim, output_dim]
        Sparse matrix representing the fixed random hash vectors `h` and `s` 
        (e.g., the return value of `kernelpooling._generate_sketch_matrix`.)
    x : array, shape (N, input_dim)
        Array of input feature vectors from which to compute CSketch representation

    Returns
    -------
    C_x : array, shape (output_dim,)
        The Count Sketch vector of `x`, the given the `sketch_matrix`
    '''

    raise NotImplementedError()

def _estimate_gamma(X_train):
    # check input form
    assert isinstance(X_train, np.ndarray), 'X_train must be a numpy array of feature vectors'
    assert X_train.ndim in [3,4], 'X_train must be a 3D or 4D array of shape (batch,...,C)'
    if X_train.ndim == 4:
        X_train.reshape(X_train.shape[0], -1, X_train.shape[-1])
    
    # compute mean inner product
    pair_count = 0
    inner_sum = 0
    for i in range(X_train.shape[0]):
        dots = X_train[i].dot(X_train[i].T)
        pair_count += dots.size
        inner_sum += dots.sum()

    # return the reciprocal
    return pair_count / inner_sum


class AlphaInitializer():
    '''Initializer object for setting initial composition_weights given `gamma`. 
    Following the Taylor series expansion of the RBF kernel:
        K(x, y) = Sum_i(beta * \frac{(2*gamma)^i}{i!})
    We assume that input vectors are L2-normalized, in which case we have:
        (alpha_i)^2 = exp(-2*gamma)*\frac{(2*gamma)^i}{i!}
    '''
    def __init__(self, gamma):
        self.gamma = gamma

    def __call__(self, shape, dtype=float):
        assert len(shape)==1, 'Only use AlphaInitializer on 1D weights'
        gam2 = np.array([2*self.gamma]*shape[0])
        beta = np.exp(-gam2)
        numer = np.power(gam2, range(gam2.size))
        denom = np.array([factorial(i) for i in range(gam2.size)])
        alpha = np.sqrt(beta * numer / denom)
        return K.variable(alpha)


class KernelPooling(Layer):
    '''Kernel Pooling layer with learnable composition weights. Takes convolution output volume as input, 
    outputs a Taylor series kernel of order `p`. By default the weights are initialized to approximate
    the Gaussian RBF kernel. See the paper for more detailed exposition. Kernel vectors are average
    pooled over all spatial locations (h_i, w_j).

    `output_shape=(batches,1+C+Sum(d_2,...,d_p))`, for `input_shape=(batches,H,W,C)`. This implementation 
    follows the paper in assuming that `d_i=d_bar` for all `i>=2`, so `d=1+C+(p-1)*d_i`.

    Parameters
    ----------
    p : int, optional
        Order of the polynomial approximation, default=3
    d_i : int, optional
        Dimensionality of output features of each order {2,...,p}, default=4096.
    seed : int, optional
        Random seed for generating sketch matrices. 
        If given, will use range(seed, seed+p) for `h`, and range(seed+p, seed+2p) for `s`.
    gamma : float, optional
        RBF kernel approximation parameter, default=1e-4.
    X_train : array, optional
        Training set of features from which to estimate gamma s.t. kernel closely approximates RBF. 
        If provided, will use reciprocal of mean inner product values. Otherwise, default used.

    # TODO : 
    '''

    def __init__(self, p=3, d_i=4096, seed=0, gamma=1e-4, X_train=None, **kwargs):
        self.p = p
        self.d_i = d_i
        self.h_seed = seed
        self.s_seed = seed + p
        if X_train is not None:
            self.gamma = _estimate_gamma(X_train)
        else:
            self.gamma = gamma
        super(KernelPooling, self).__init__(**kwargs)

    def build(self, input_shape):
        #self._shapecheck(input_shape)

        # Initialize composition weights, RBF approximation
        alpha_init = AlphaInitializer(self.gamma)
        self.alpha = self.add_weight(name='composition_weights',
                                    shape=(self.p+1,),
                                    initializer=alpha_init,
                                    trainable=True)

        # Generate sketch matrices, need `p` sets of {h_t, s_t}
        self.C = input_shape[-1]
        self.sketch_matrices = []
        h_seeds = range(self.h_seed, self.h_seed+self.p)
        s_seeds = range(self.s_seed, self.s_seed+self.p)
        for hs, ss in zip(h_seeds,s_seeds):
            np.random.seed(hs)
            h_t = np.random.randint(self.d_i, size=self.C)
            np.random.seed(ss)
            s_t = 2*np.random.randint(2, size=self.C)-1
            self.sketch_matrices.append(_generate_sketch_matrix(h_t, s_t, self.d_i))
            

    def call(self, x):
        input_dims = K.shape(x)
        
        # zeroth and first order terms
        zeroth = self.alpha[0]*K.ones((input_dims[0],1))
        first  = self.alpha[1]*tf.reduce_mean(x, axis=[1,2])

        # flatten to feature vectors
        x_flat = K.reshape(x, (-1, self.C))
        
        # Compute the Count Sketches C_t over feature vectors
        sketches = []
        for t in range(self.p):
            sketches.append(tf.transpose(tf.sparse_tensor_dense_matmul(self.sketch_matrices[t],
                                         x_flat, adjoint_a=True, adjoint_b=True)))
            
        # stack and reshape [(b*h*w, d_i)], len=p --> (b, h*w, p, d_i) 
        x_sketches = K.reshape(K.stack(sketches, axis=-2), (input_dims[0], -1, self.p, self.d_i))
        
        # Compute fft (operates on inner-most axis)
        x_fft = _fft(tf.complex(real=x_sketches, imag=K.zeros_like(x_sketches)), False, 128)
        
        # Cumulative product along order dimension, discard first order
        x_fft_cp = K.cumprod(x_fft, axis=-2)[:, :, 1:, :]
        
        # Inverse fft, avg pool over spatial locations
        x_ifft = tf.reduce_mean(tf.real(_ifft(x_fft_cp, False, 128)), axis=1)
        
        # Apply weights over orders p >= 2
        x_p = x_ifft*K.reshape(self.alpha[2:], (1,self.p-1,1))
        
        # Concatenate to full order-p kernel approximation vector
        phi_x = K.concatenate([zeroth, first, K.reshape(x_p, (input_dims[0],-1))])
        
        # Return the transformed + l2-normed kernel vector
        phi_x = tf.multiply(tf.sign(phi_x),tf.sqrt(tf.abs(phi_x)+1e-12))

        return tf.nn.l2_normalize(phi_x, axis=-1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], 1+input_shape[-1]+(self.p-1)*self.d_i)



In [124]:
from texture.cnn import keras_apps
from keras import layers, models

In [111]:
resnet = keras_apps['resnet50'](include_top=False, input_shape=(224,224,3))

In [ ]:
resnet.summary()

In [136]:
x = resnet.output
x = KernelPooling(p=4)(x)
pred = layers.Dense(47, activation='softmax')(x)

kernel_cnn = models.Model(inputs=resnet.inputs, outputs=pred)

kernel_cnn.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [108]:
p = 3
h_seed = 1
s_seed = 1+p
C = 512
d_i = 128

alpha = tf.constant(list(range(1,p+2)), dtype=tf.float32)

elems = tf.random_uniform([4,25,C], dtype=tf.float32, seed=106)

zeroth = alpha[0]*K.ones((4,1))
first = alpha[1]*tf.reduce_mean(elems, axis=1)

sketch_matrices = []
h_seeds = range(h_seed, h_seed+p)
s_seeds = range(s_seed, s_seed+p)
for hs, ss in zip(h_seeds,s_seeds):
    np.random.seed(hs)
    h_t = np.random.randint(d_i, size=C)
    np.random.seed(ss)
    s_t = 2*np.random.randint(2, size=C)-1
    sketch_matrices.append(_generate_sketch_matrix(h_t, s_t, d_i))

def compute_sketchs(X_3D):
    sketches = []
    flat_X = tf.reshape(X_3D, [-1, C])
    for i in range(p):
        #sketches.append(flat_X)
        sketches.append(tf.transpose(tf.sparse_tensor_dense_matmul(sketch_matrices[i],
                                    flat_X, adjoint_a=True, adjoint_b=True)))
    return sketches

X_sketches = compute_sketchs(elems)
X_rshape = tf.reshape(tf.stack(X_sketches, axis=-2), (4, 25, p, d_i))
X_fft = tf.fft(tf.complex(real=X_rshape, imag=tf.zeros_like(X_rshape)))
X_cprod = tf.cumprod(X_fft, axis=-2)[:,:,1:,:]

X_p = tf.reduce_mean(tf.real(tf.ifft(X_cprod)), axis=1)*K.reshape(alpha[2:], (1,2,1))

p_23 = K.reshape(X_p, (4, -1))

f_vector = K.concatenate([zeroth, first, p_23])

#X_reshape = tf.reshape(X_stack, (2,3,-1,C))

#alternates = tf.map_fn(lambda x: tf.cumsum(x), c_elems, dtype=tf.complex64)
#alternates = tf.stack(list(alternates), -1)
#x = tf.random_uniform([2,3,5], maxval=10, dtype=tf.int32, seed=100)
#reshaped_x = tf.reshape(x, [-1, 5])

with tf.Session() as sess:
    print(sess.run(tf.shape(zeroth)))
    print(sess.run(tf.shape(first)))
    print(sess.run(tf.shape(X_cprod)))
    print(sess.run(tf.shape(X_p)))
    print(sess.run(tf.shape(f_vector)))
    

    

[4 1]
[  4 512]
[  4  25   2 128]
[  4   2 128]
[  4 769]


In [98]:
test_elems = tf.constant(np.random.randint(0, 10, (3,2,3)).tolist(), dtype=tf.int32)
test_alpha = tf.constant([1,10], dtype=tf.int32)

mult = test_elems*tf.reshape(test_alpha,[1,2,1])

with tf.Session() as sess:
#    sess.run(tf.initialize_all_variables())
    print(sess.run(test_elems))
    print(sess.run(test_alpha))
    print(sess.run(mult))
    


[[[9 5 9]
  [2 5 8]]

 [[4 6 7]
  [9 9 1]]

 [[4 5 1]
  [3 0 1]]]
[ 1 10]
[[[ 9  5  9]
  [20 50 80]]

 [[ 4  6  7]
  [90 90 10]]

 [[ 4  5  1]
  [30  0 10]]]


In [95]:
x = np.array([1,2,3])
x.to

In [ ]:
np.real(np.fft.fft(np.complex(np.array([0.01501954, 0.7412647,  0.3279165,  0.80562544, 0.2800964])), n =5))

In [ ]:
from scipy.fftpack import fft

vals = [0.01501954, 0.7412647,  0.3279165,  0.80562544, 0.2800964]
carr = np.array([np.complex64(v) for v in vals])
fft(carr)

In [67]:
a = np.random.rand(2,3,4,5)
cp = np.cumprod(a, axis=-2)

a[0,0,:2,:], cp[0,0,:2,:]


(array([[0.11760679, 0.57039609, 0.73365592, 0.79658239, 0.13625902],
        [0.47684169, 0.41103673, 0.39908856, 0.83066492, 0.18771481]]),
 array([[0.11760679, 0.57039609, 0.73365592, 0.79658239, 0.13625902],
        [0.05607982, 0.23445375, 0.29279368, 0.66169305, 0.02557784]]))

In [68]:
0.11761*0.476841

0.05608127001

In [131]:
f = np.random.randint(0, 10, (2,3))

dots = f.dot(f.T)

print(f, '\n', f.T, '\n', dots)

dots.shape

[[5 2 8]
 [7 7 6]] 
 [[5 7]
 [2 7]
 [8 6]] 
 [[ 93  97]
 [ 97 134]]


(2, 2)

In [133]:
5*7 + 2*7 + 8*6

97